In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random # Ignore it 
import sys # ignore it
import cv2  # Computer Vision Library

#import matplotlib # graying of the images
#from PIL import image as pil
from subprocess import check_output # accuracy and quality

import matplotlib.pyplot as plt
from datetime import datetime

from skimage.morphology import disk
from skimage.filters.rank import gradient
#from skimage import data
#from skimage.color import rgb2gray #Grayscale
#!pip install opencv-python

#from imutils import paths
#import argparse
#!pip install imutils

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from keras.models import Sequential # Easy-to-use DL Library
#from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
#from keras.optimizers import Adam # Labeling techniques used for medical data like images
#from sklearn.model_selection import train_test_split # bunch of ML libraries, train / test split of fundus images
#from keras.utils import to_categorical
#print(os.listdir("/content/drive/My Drive/Education/BlueBoxAI/Retina Images/a. Training Set"))

#ORIGINAL IMAGES #print(os.listdir("/content/drive/My Drive/Education/BlueBoxAI/DRD_work/Input/A. Segmentation/1. Original Images/a. Training Set"))
print(os.listdir("/content/drive/My Drive/a. Training Set-20190302T161936Z-001.zip (Unzipped Files)/a. Training Set"))

NUM_CLASSES = 5
# we need images of same size so we convert them into the size
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (HEIGHT, WIDTH, DEPTH)
# initialize number of epochs to train for, initial learning rate and batch size
EPOCHS = 15
INIT_LR = 1e-3
BS = 32
#global variables
ImageNameDataHash = {}
uniquePatientIDList = []


In [0]:
#define a function to get the images from the folders and change the size
def readTrainData(trainDir):
    global ImageNameDataHash
    # loop over the input images
    images = os.listdir(trainDir)
    print("Number of files in " + trainDir + " is " + str(len(images)))
    for imageFileName in images:
#       if (imageFileName == "IDRiD_50.jpg"):  
        #joining the trainDir with image name using / as a separator to get full path for the imagefile
        imageFullPath = os.path.join(os.path.sep, trainDir, imageFileName)
        
        # load the image, pre-process it, and store it in the data list
        img = cv2.imread(imageFullPath)
        img = cv2.resize(img, (4000, 2500)) 
        imgbgr = img
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        arr = img_to_array(img)  # Numpy array with shape (233,233,3)
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
       
        if (dim1 < 128 or dim2 < WIDTH or dim3 < DEPTH):
            print("Error image dimensions are less than expected "+str(arr.shape))
        
        ratio = dim1 / dim2
        HEIGHT = int(WIDTH*ratio)
        
        #.....dont resize for now.....arr = cv2.resize(arr, (WIDTH, HEIGHT)) #Numpy array with shape (HEIGHT, WIDTH,3)
        #print(arr.shape) # 128,128,3
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        #if (dim1 != HEIGHT or dim2 != WIDTH or dim3 != DEPTH):
            #print("Error after resize, image dimensions are not equal to expected "+str(arr.shape))
        #print(type(arr))
        # scale the raw pixel intensities to the range [0, 1] - TBD TEST
        arr = np.array(arr, dtype="float") / 255.0
        #imageFileName = imageFileName.replace('.jpeg','')
        ImageNameDataHash[str(imageFileName)] = np.array(arr)
        print(str(dim1) + "," + str(dim2) + "," + str(dim3))
        
        plt.imshow(img)
        plt.show()
        
        #retina_gray = rgb2gray(arr)
        #plt.imshow(retina_gray) #Show grayscale image
        #plt.show()
        #import opencv
        
        #Reduce noise by blurring with a Gaussian filter ( kernel size = 3 )
        retinaGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
        plt.imshow(retinaGray)
        plt.show()
        
        binary = cv2.GaussianBlur(retinaGray,(5,5),0)
        
        selection_element = disk(5) # matrix of n pixels with a disk shape
        retina_sharpness = gradient(retinaGray, selection_element)
        sharpImg = np.array(retinaGray, dtype="float") / 255.0
        plt.imshow(sharpImg, cmap="viridis")
        plt.axis('off')
        plt.colorbar()
        plt.show()
      
        
        # compute the Laplacian of the image and then return the focus
        # measure, which is simply the variance of the Laplacian
        fm = cv2.Laplacian(retinaGray, cv2.CV_64F).var()
        text = "Good"

        # if the focus measure is less than the supplied threshold,
        # then the image should be considered "blurry"
        if fm > 50:
          text = "Bad"
        print(str(imageFileName) + ": " + text + " " + str(fm))
        
        
    #ENHANCE THE CONTRAST OF THE IMAGE TO OVERCOME LIGHTING ISSUES
    
    #   convert BGR image into the L*a*b* color space.
    lab = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2LAB)
    #Split the LAB format image into three channels
    lab_planes = cv2.split(lab)
    # Perform CLAHE on the channel
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    #perform CLAHE on L channel
    lab_planes[0] = clahe.apply(lab_planes[0])
    #Merge the image back with new L channel
    lab = cv2.merge(lab_planes)
    #Convert the LAB image back to RGB
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    plt.imshow(rgb)
    
    
    return

In [0]:
print("Loading images at..."+ str(datetime.now()))
sys.stdout.flush()
#readTrainData("/kaggle/working/../input/")
# readTrainData("/content/drive/My Drive/Education/BlueBoxAI/Retina Images/a. Training Set")

#DISPLAY IMAGES
#ORIGINAL IMAGES #readTrainData("/content/drive/My Drive/Education/BlueBoxAI/DRD_work/Input/A. Segmentation/1. Original Images/a. Training Set")
readTrainData("/content/drive/My Drive/a. Training Set-20190302T161936Z-001.zip (Unzipped Files)/a. Training Set")

print("Loaded " + str(len(ImageNameDataHash)) + " images at..."+ str(datetime.now())) # 1000